In [0]:
#@title IMPORT STUFF
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.optimize import curve_fit
import glob
import pickle
import urllib
import datetime
import scipy
from scipy import stats
from pylab import *
import os
from matplotlib.colors import ListedColormap
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

In [2]:
#@title CHANGE DIRECTORY

##### DANIELLA's DIRECTORY 
os.chdir('/content/drive/My Drive/HP Rivendell')

##### DAVID's DIRECTORY
# os.chdir('/content/drive/My Drive/research/HP Rivendell')

FileNotFoundError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def getFlow(site,start,stop):
    url = 'https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=' + site + '&referred_module=sw&period=&begin_date='+start+'&end_date='+stop
    df = pd.read_csv(url, header=31, delim_whitespace=True)
    df.columns = ['usgs', 'site', 'datetime', 'q', 'a']
    df.index = pd.to_datetime(df.datetime)
    df = df[['q']]
    df.q = df.q.astype(float, errors='ignore')
    return df

In [7]:
rainfall = pd.read_csv('data/elder_rainfall.csv', index_col=0,parse_dates=True)
elder = getFlow('11475560','2000-01-01','2019-12-30')
elder = elder[['q']]*2.447e+12 # convert to mm^3/day
area = 16.8 * 1e+12 # convert to mm/day
elder = elder/area


# get water year
elder['wy'] = [item.year+1 if (item.month<=12)&(item.month>=10) else item.year for item in elder.index]
rainfall['wy'] = [item.year+1 if (item.month<=12)&(item.month>=10) else item.year for item in rainfall.index]
elder = elder.loc['2000-10-01':'2018-12-31']
rainfall = rainfall.loc['2000-10-01':'2018-12-31']

FileNotFoundError: ignored

In [6]:
years = elder.index.year.unique()
wyrain = []
lowflow = []
yearplot = []
lowflowdate = []
for year in years[1:]:
    wystart = str(year-1) + '-10-01'
    wyend = str(year) + '-9-1'
    sd = str(year) + '-05-01'
    ed = str(year) + '-12-01'
    wyrain.append(rainfall.ppt.loc[wystart:wyend].sum())
    lowflow.append(elder.q.loc[sd:ed].min())
    lowflowdate.append(elder.q.loc[sd:ed].idxmin())
    yearplot.append(year)
    
slope, intercept, r_value, p_value, std_err = stats.linregress(np.array(wyrain),np.array(lowflow))
f,axs = plt.subplots(1,2,figsize=(10,4), sharey=True)
cmap = ListedColormap(sns.color_palette('tab20',len(yearplot)).as_hex()).reversed()
pal = np.flip(sns.color_palette('tab20',len(yearplot)).as_hex(), axis=0)
ax = axs[1]
sp = ax.scatter(wyrain, lowflow,c=yearplot,cmap=cmap,s=75,edgecolors='k')
bounds = np.array([i for i in yearplot + [yearplot[-1]+1]])-0.5
plt.colorbar(sp, ticks=[i for i in yearplot if i%1==0], boundaries=bounds)
titlestr = 'Annual low flow vs. water year precipitation'
titlestr += '\nR$^2$ = %.2f, p-value on slope = %.2f'%(r_value**2, p_value)
ax.set_title(titlestr, fontsize=10)
ax.set_xlabel('Water year precipitation [mm]')
ax.set_yscale('log')
ax = axs[0]
tp = range(2009,2019)
labels = []
handles = []
for i,year in enumerate(years[1:]):
    if year in tp:
        sd = str(year) + '-4-01'
        ed = lowflowdate[i]
        toplot = elder.q.loc[sd:ed]
        x = range(len(toplot))
        h, = ax.plot(x,toplot,c=pal[i])
        handles.append(h)
        labels.append(year)        
        ax.scatter(x[-1],toplot[-1],c=pal[i],s=75,edgecolors='k',zorder=100)
    else:
        continue
ax.legend(labels, fontsize=10,ncol=2)
ax.set_ylabel('Runoff [mm/day]')
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.set_title('Summer recessions, Elder Creek', fontsize=10)
ax.set_xlabel('Days since April 1')
f.tight_layout()
# f.savefig('../figures/lowflow_wyprecip.pdf')

NameError: ignored